To do:
1. double check logstic convergence issues
2. do 5-fold

In [1]:
import os
import json
import pandas as pd
import numpy as np
from sklearn.metrics import make_scorer
from sklearn.linear_model import RidgeCV, LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
from rca import mcfadden_r2_binary, mcfadden_r2_multiclass, process_categorical, best_logistic_solver, checker, k_fold_cross_val
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Loading Data

In [2]:
# Pulling and standardising embeddings
embeds = {}
for f_name in os.listdir('../../data/processed/pulled_embeds/'):
    embed = pd.read_pickle(f'../../data/processed/pulled_embeds/{f_name}')
    embed_name = f_name.split('.')[0]
    embeds[embed_name] = (embed - embed.mean()) / embed.std()

{name: embed.shape for name, embed in embeds.items()}

{'SVD_sim_rel': (6001, 300),
 'eye_motor_response': (7469, 6),
 'fastText_Wiki_News': (81696, 300),
 'fastTextSub_OpenSub': (72511, 300),
 'SGSoftMaxEncoder_SWOW': (11781, 300),
 'PPMI_SVD_SWOW': (11781, 300),
 'GloVe_Twitter': (48587, 200),
 'GloVe_Wikipedia': (68921, 300),
 'GloVe_CommonCrawl': (88408, 300),
 'fMRI_text_cos_align': (1200, 1000),
 'fMRI_speech_hyper_align': (579, 6),
 'THINGS': (1562, 49),
 'CBOW_GoogleNews': (79253, 300),
 'EEG_speech': (1591, 130),
 'BERT_last4_Book_Wikipedia': (24761, 1024),
 'compo_attribs': (534, 62),
 'morphoNLM': (50480, 50),
 'fMRI_text_hyper_align': (1200, 1000),
 'norms_sensorimotor': (36851, 11),
 'EEG_text': (3342, 104),
 'LexVec_CommonCrawl': (87613, 300),
 'fMRI_speech_cos_align': (579, 6),
 'fastText_CommonCrawl': (88953, 300),
 'spherical_text_Wikipedia': (58987, 300),
 'BERT_first4_Book_Wikipedia': (24761, 1024),
 'SGSoftMaxDecoder_SWOW': (25435, 300)}

In [2]:
norms = pd.read_csv('../../data/raw/psychNorms.zip', index_col=0, compression='zip')
norm_metadata = pd.read_csv('../../data/raw/psychNorms_metadata.csv', index_col=0)
norm_metadata['associated_embed'] = norm_metadata['associated_embed'].astype(str)
norms

/var/folders/st/5gbrfvhn687dxwjl5_xg21t40000gq/T/ipykernel_47014/1354352021.py:1: DtypeWarning: Columns (22,23,170) have mixed types. Specify dtype option on import or set low_memory=False.
  norms = pd.read_csv('../../data/final/norms.csv', index_col=0)


,Freq_HAL,Freq_KF,Freq_SUBTLEXUS,Freq_SUBTLEXUK,Freq_Blog,Freq_Twitter,Freq_News,Freq_CobW,Freq_CobS,CD_SUBTLEXUS,...,iconicity_winter_2017,living_vanarsdall,thought_vanarsdall,reproduction_vanarsdall,person_vanarsdall,goals_vanarsdall,movement_vanarsdall,concreteness_vanarsdall,familiarity_vanarsdall,imageability_vanarsdall
'em,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.3617,1.9138,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'neath,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
're,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.9031,1.6335,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'shun,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'tis,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.4771,0.6021,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trappy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vocalise,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
listened..to.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
spoke..to.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
norms.count().sum()

2851952

In [2]:
with open('../../data/raw/dtype_to_embed.json', 'r') as f:
    type_to_embed = json.load(f)
    
embed_to_type = {}
for type, embeds in type_to_embed.items():
    for embed in embeds:
        embed_to_type[embed] = type
embed_to_type

{'CBOW_GoogleNews': 'text',
 'fastText_CommonCrawl': 'text',
 'fastText_Wiki_News': 'text',
 'fastTextSub_OpenSub': 'text',
 'GloVe_CommonCrawl': 'text',
 'GloVe_Twitter': 'text',
 'GloVe_Wikipedia': 'text',
 'LexVec_CommonCrawl': 'text',
 'morphoNLM': 'text',
 'spherical_text_Wikipedia': 'text',
 'eye_tracking': 'brain',
 'EEG_speech': 'brain',
 'EEG_text': 'brain',
 'fMRI_speech_hyper_align': 'brain',
 'fMRI_text_hyper_align': 'brain',
 'microarray': 'brain',
 'PPMI_SVD_SWOW': 'behavior',
 'SGSoftMaxInput_SWOW': 'behavior',
 'SGSoftMaxOutput_SWOW': 'behavior',
 'PPMI_SVD_SOUTH_FLORIDA': 'behavior',
 'THINGS': 'behavior',
 'feature_overlap': 'behavior',
 'norms_sensorimotor': 'behavior',
 'compo_attribs': 'behavior',
 'SVD_sim_rel': 'behavior'}

## Cross Validation

In [48]:
# Ridge
alphas = np.logspace(-3, 3, 6)
ridge = RidgeCV(alphas=alphas)

# Logistic hyperparameters
Cs = 1 / alphas
inner_cv = 5
penalty = 'l2'

# Scorers
binary_scorer = make_scorer(mcfadden_r2_binary, needs_proba=True, greater_is_better=True)
multiclass_scorer = make_scorer(mcfadden_r2_multiclass, needs_proba=True, greater_is_better=True)

# outer_cv setting 
outer_cv, n_jobs = 5, 8

In [49]:
# RCA
rca = []
for embed_name in tqdm(embeds.keys()):
    embed = embeds[embed_name]
    
    to_print = []
    for norm_name in tqdm(norms.columns, desc=embed_name):
        
        # Aligning data
        norm = norms[norm_name].dropna()
        embed, norm = embed.align(norm, axis=0, join='inner')
        
        # Checking norm dtype 
        norm_dtype = norm_metadata.loc[norm_name, 'type']
        
        # Solvers, scoring, estimators
        if norm_dtype in ['binary', 'multiclass']:
            embed, norm = process_categorical(embed, norm, outer_cv, inner_cv)
            
            # may have switched form multi to bin after processing
            norm_dtype = 'binary' if len(norm.unique()) == 2 else 'multiclass'
            
            # Cross validation settings for logistic regression
            solver = best_logistic_solver(embed, norm_dtype)
            
            # Defining logistic regression 
            estimator = LogisticRegressionCV(
                Cs=Cs, penalty=penalty, cv=StratifiedKFold(inner_cv),
                solver=solver, n_jobs=8
            )
        else: # continuous
            estimator, scoring = ridge, 'r2'

            
        # Cross validation
        check = checker(
            embed_name, norm, norm_dtype, norm_metadata, outer_cv, norm_name
        )
        if check == 'pass':
            r2_mean, r2_sd = k_fold_cross_val(estimator, embed, norm, norm_dtype, outer_cv, n_jobs)
        else:
            r2 = np.nan
            
        # Saving
        train_n = int(((outer_cv - 1) / outer_cv) * len(embed))
        p = embed.shape[1]
        embed_type = embed_to_type[embed_name]
        rca.append([embed_name, embed_type, norm_name, train_n, p, r2, check])
        
        to_print.append([norm_name, train_n, r2, check])

    
    to_print = (
        pd.DataFrame(to_print, columns=['norm' , 'train_n', 'r2', 'check'])
        .sort_values('r2', ascending=False).head(10)
    )
    print(to_print)

rca = pd.DataFrame(rca, columns=['embed', 'embed_type', 'norm', 'train_n', 'p', 'r2', 'check'])
rca.to_csv('../../data/final/rca_test.csv', index=False)
rca

  0%|          | 0/26 [00:00<?, ?it/s]

-------------------
SVD_sim_rel
-------------------


  0%|          | 0/281 [00:00<?, ?it/s]

/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The ma

                        norm  train_n        r2 check
72       Emot_Assoc_Surprise     3023  0.737305  pass
66        Emot_Assoc_Disgust     3023  0.681822  pass
68            Emot_Assoc_Joy     3023  0.669039  pass
71        Emot_Assoc_Sadness     3023  0.613820  pass
65   Emot_Assoc_Anticipation     3023  0.608344  pass
64          Emot_Assoc_Anger     3023  0.606539  pass
122             Music_Binder      283  0.581443  pass
67           Emot_Assoc_Fear     3023  0.527708  pass
73          Emot_Assoc_Trust     3023  0.505152  pass
272        living_vanarsdall      697  0.376176  pass
-------------------
eye_motor_response
-------------------


...

/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(


  0%|          | 0/281 [00:00<?, ?it/s]

/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The ma

                        norm  train_n        r2 check
72       Emot_Assoc_Surprise     9420  0.778104  pass
68            Emot_Assoc_Joy     9420  0.761546  pass
65   Emot_Assoc_Anticipation     9420  0.688682  pass
66        Emot_Assoc_Disgust     9420  0.680686  pass
71        Emot_Assoc_Sadness     9420  0.662569  pass
64          Emot_Assoc_Anger     9420  0.643570  pass
73          Emot_Assoc_Trust     9420  0.612082  pass
190                sad_zupan      356  0.597898  pass
214         social_des_britz      376  0.596224  pass
162          goals_wilkowski      661  0.585780  pass


,embed,norm,train_n,p,r2,check
0,SVD_sim_rel,Freq_HAL,4506,300,0.089362,pass
1,SVD_sim_rel,Freq_KF,3776,300,0.036674,pass
2,SVD_sim_rel,Freq_SUBTLEXUS,4450,300,0.156546,pass
3,SVD_sim_rel,Freq_SUBTLEXUK,4472,300,0.060241,pass
4,SVD_sim_rel,Freq_Blog,4652,300,0.098733,pass
...,...,...,...,...,...,...
7301,SGSoftMaxDecoder_SWOW,goals_vanarsdall,959,300,0.403416,pass
7302,SGSoftMaxDecoder_SWOW,movement_vanarsdall,959,300,0.345155,pass
7303,SGSoftMaxDecoder_SWOW,concreteness_vanarsdall,959,300,0.132158,pass
7304,SGSoftMaxDecoder_SWOW,familiarity_vanarsdall,959,300,0.081765,pass
